## Deliverable 2. Create a Customer Travel Destinations Map.

In [41]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [42]:
# 1. Import the WeatherPy_database.csv file. 
path_to_file = "../Weather_Database/WeatherPy_Database.csv"
city_data_df = pd.read_csv(path_to_file)
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Turbat,PK,26.0023,63.0440,74.44,74,43,3.00,light rain
1,1,Stepnyak,KZ,52.8339,70.7803,27.07,90,100,8.70,overcast clouds
2,2,Dakar,SN,14.6937,-17.4441,76.08,64,93,6.78,overcast clouds
3,3,Arinos,BR,-15.9169,-46.1056,70.84,67,34,1.66,light rain
4,4,Paamiut,GL,61.9940,-49.6678,16.45,81,21,7.90,few clouds


In [43]:
city_data_df.dtypes

City_ID          int64
City            object
Country         object
Lat            float64
Lng            float64
Max Temp       float64
Humidity         int64
Cloudiness       int64
Wind Speed     float64
Description     object
dtype: object

In [44]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip?"))
max_temp = float(input("What is the maximum temperature you would like for your trip?"))

What is the minimum temperature you would like for your trip?65
What is the maximum temperature you would like for your trip?95


In [45]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]

preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Turbat,PK,26.0023,63.0440,74.44,74,43,3.00,light rain
2,2,Dakar,SN,14.6937,-17.4441,76.08,64,93,6.78,overcast clouds
3,3,Arinos,BR,-15.9169,-46.1056,70.84,67,34,1.66,light rain
6,6,Ndende,GA,-2.4008,11.3581,72.00,98,100,2.77,overcast clouds
7,7,Vila Velha,BR,-20.3297,-40.2925,69.94,88,75,4.61,broken clouds
9,9,Hithadhoo,MV,-0.6000,73.0833,81.43,76,46,12.59,scattered clouds
10,10,Arraial Do Cabo,BR,-22.9661,-42.0278,69.03,68,25,5.21,scattered clouds
13,13,Kavaratti,IN,10.5669,72.6420,80.87,75,4,3.69,clear sky
16,16,Ribeira Grande,PT,38.5167,-28.7000,65.77,79,100,15.01,overcast clouds
25,25,Alta Floresta,BR,-9.8756,-56.0861,75.96,64,36,1.70,scattered clouds


In [46]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID        338
City           338
Country        337
Lat            338
Lng            338
Max Temp       338
Humidity       338
Cloudiness     338
Wind Speed     338
Description    338
dtype: int64

In [47]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_travel_cities = preferred_cities_df.dropna()
clean_travel_cities.count()

City_ID        337
City           337
Country        337
Lat            337
Lng            337
Max Temp       337
Humidity       337
Cloudiness     337
Wind Speed     337
Description    337
dtype: int64

In [48]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_travel_cities[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Turbat,PK,74.44,light rain,26.0023,63.0440,
2,Dakar,SN,76.08,overcast clouds,14.6937,-17.4441,
3,Arinos,BR,70.84,light rain,-15.9169,-46.1056,
6,Ndende,GA,72.00,overcast clouds,-2.4008,11.3581,
7,Vila Velha,BR,69.94,broken clouds,-20.3297,-40.2925,
9,Hithadhoo,MV,81.43,scattered clouds,-0.6000,73.0833,
10,Arraial Do Cabo,BR,69.03,scattered clouds,-22.9661,-42.0278,
13,Kavaratti,IN,80.87,clear sky,10.5669,72.6420,
16,Ribeira Grande,PT,65.77,overcast clouds,38.5167,-28.7000,
25,Alta Floresta,BR,75.96,scattered clouds,-9.8756,-56.0861,


In [49]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params = params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found. Skipping...")

print("Hotel Search Complete.")

Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not foun

In [95]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [98]:
# 8a. Create the output File (CSV)
output_file_1 = "./WeatherPy_vacation1.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_file_1, index_label = "City_ID")

In [99]:
# Error correction: Pandas DF wasn't reading NaN values.
path = "./WeatherPy_vacation1.csv"
clean_df_v2 = pd.read_csv(path)
clean_df_v2.head()

,City_ID,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,0,Turbat,PK,74.44,light rain,26.0023,63.0440,Mubashar Quarters
1,2,Dakar,SN,76.08,overcast clouds,14.6937,-17.4441,Novotel Dakar
2,3,Arinos,BR,70.84,light rain,-15.9169,-46.1056,casa do bruno
3,6,Ndende,GA,72.00,overcast clouds,-2.4008,11.3581,NaN
4,7,Vila Velha,BR,69.94,broken clouds,-20.3297,-40.2925,Hotel Vitória Palace


In [100]:
# Cleaning and importing new DF to database. 
cleanest_df = clean_df_v2.dropna()
cleanest_df.drop("City_ID", inplace=True, axis=1)

C:\Users\mguzman\AppData\Local\Temp\ipykernel_9832\210209511.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleanest_df.drop("City_ID", inplace=True, axis=1)


In [101]:
cleanest_df.head()

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Turbat,PK,74.44,light rain,26.0023,63.0440,Mubashar Quarters
1,Dakar,SN,76.08,overcast clouds,14.6937,-17.4441,Novotel Dakar
2,Arinos,BR,70.84,light rain,-15.9169,-46.1056,casa do bruno
4,Vila Velha,BR,69.94,broken clouds,-20.3297,-40.2925,Hotel Vitória Palace
5,Hithadhoo,MV,81.43,scattered clouds,-0.6000,73.0833,Scoop Guest House


In [102]:
# Create the output File (CSV)
output_file = "./WeatherPy_vacation.csv"
# Export the City_Data into a csv
cleanest_df.to_csv(output_file, index_label="City_ID")

In [103]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City Name</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in cleanest_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = cleanest_df[["Lat", "Lng"]]

In [88]:
# 11a. Add a marker layer for each city to the map. 
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center = (30.0,31.0), zoom_level = 1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))